In [ ]:
import ee
import geemap

In [ ]:
Map = geemap.Map()
Map.clear_layers()
Map.add_basemap('HYBRID')
Map

In [ ]:
def imagens(datainicial,datafinal,nuvem):
    Map.clear_layers()
    Map.add_basemap('HYBRID')
    colecao = ee.ImageCollection('COPERNICUS/S2_SR')\
    .filterDate(datainicial,datafinal)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',nuvem))\
    .filterBounds(Map.user_roi.getInfo())\
    .select(['B4','B3', 'B2'])\
    .sort('system:time_start', False)
    
    
    # loop aviso
    contador = 0
    if colecao.size().getInfo() > 15:
        contador = 15
        print(f'ATENÇÃO, O NÚMERO TOTAL DE IMAGENS NA COLEÇÃO PARA O PERÍODO É DE : {colecao.size().getInfo()} \n') 
        print('15 IMAGENS É O LIMITE MÁXIMO DE VISUALIZAÇÃO. ALGUMAS IMAGENS NÃO SERÃO APRESENTADAS \n')
    elif colecao.size().getInfo() == 0:
        print('NÃO FORAM ENCONTRADAS IMAGENS PARA O PERÍODO')
    else:
        print(f'O NÚMERO TOTAL DE IMAGENS NA COLEÇAO PARA O PERÍODO É DE : {colecao.size().getInfo()} \n') 
        contador = colecao.size().getInfo()
    
    # Transformando imagecollection em lista
    collist = colecao.toList(colecao.size())
    
    # parametros de visualizacao
    vizparam = {
    'min': 0,
    'max': 2000,
    'bands': ['B4', 'B3','B2'],
    'gamma': 1.5}

    # loop de display 
    cont = 0
    for i in range(contador):
        cont += 1
        print(colecao.aggregate_array('PRODUCT_ID').getInfo()[i])
        
        image = ee.Image(collist.get(i))
        
        date = image.date().format('yyyy-MM-dd').getInfo()
        
        Map.addLayer(ee.Image(collist.get(i)),vizparam, str(date + '_' + str(cont))) 
    
    Map.centerObject(ee.Image(collist.get(0)), 9)
    
    return Map

In [ ]:
imagens('2021-01-01','2021-05-15',10)